<a href="https://colab.research.google.com/github/ndb796/Deep-Learning-Paper-Review-and-Practice/blob/master/code_practices/TextFooler_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### <b>TextFooler: 텍스트 분류 모델 공격 실습</b>

* 특정 클래스로 분류되는 하나의 문장이 <b>다른 클래스로 인식</b>되도록 만들어 봅시다.
* <b>Black-box attack</b>: 공격자가 공격 대상 모델의 파라미터를 모르지만 <b>쿼리(query)</b>를 날려 confidence score를 구할 수 있습니다.
* <b>Wore-wise perturbing attack</b>: 문장 내 단어(word)를 하나씩 변경하는 방식으로 공격합니다.

#### <b>기본 라이브러리 설치</b>

* <b>TextFooler</b> 소스코드를 다운로드합니다.
* <b>필수적인 라이브러리</b>: Python, TensorFlow, Torch, NLTK 등을 설치</b>합니다.
* <b>[런타임]</b> → <b>[런타임 유형 변경]</b> → <b>[GPU]</b>
* Colab Notebook 재부팅이 필요하다면 <b>[Restart Runtime]</b> 버튼을 누릅니다.

In [1]:
# TextFooler 소스코드 다운로드 및 필수 라이브러리 설치 (+ 오류가 발생하지 않도록 Boto3 최신 라이브러리 설치)
!git clone https://github.com/ndb796/TextFooler
%cd TextFooler
!pip install -r requirements.txt

fatal: destination path 'TextFooler' already exists and is not an empty directory.
/content/TextFooler


In [2]:
%%capture
# Google Colab에서 실행하기 위해 추가적으로 Pattern 라이브러리 설치
!wget https://files.pythonhosted.org/packages/1e/07/b0e61b6c818ed4b6145fe01d1c341223aa6cfbc3928538ad1f2b890924a3/Pattern-3.6.0.tar.gz
!sudo tar -xvf Pattern-3.6.0.tar.gz
%cd Pattern-3.6
!python setup.py install
%cd ..

#### <b>워드 임베딩(Word Embedding) 불러오기</b>

* <b>워드 임베딩(Word Embedding)</b>을 위한 Counter Fitted Vector 텍스트 파일을 다운로드합니다.

In [10]:
!wget https://postechackr-my.sharepoint.com/:u:/g/personal/dongbinna_postech_ac_kr/ESV6s3ZdmTxLiDpzmEc2LQgBmEQF2C_0kqCsu013uq80gg?download=1 -O counter-fitted-vector.txt.zip
!unzip counter-fitted-vector.txt.zip

--2020-11-20 22:05:26--  https://postechackr-my.sharepoint.com/:u:/g/personal/dongbinna_postech_ac_kr/ESV6s3ZdmTxLiDpzmEc2LQgBmEQF2C_0kqCsu013uq80gg?download=1
Resolving postechackr-my.sharepoint.com (postechackr-my.sharepoint.com)... 40.108.156.33
Connecting to postechackr-my.sharepoint.com (postechackr-my.sharepoint.com)|40.108.156.33|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /personal/dongbinna_postech_ac_kr/Documents/Research/datasets/counter-fitted-vectors.txt.zip?originalPath=aHR0cHM6Ly9wb3N0ZWNoYWNrci1teS5zaGFyZXBvaW50LmNvbS86dTovZy9wZXJzb25hbC9kb25nYmlubmFfcG9zdGVjaF9hY19rci9FU1Y2czNaZG1UeExpRHB6bUVjMkxRZ0JtRVFGMkNfMGtxQ3N1MDEzdXE4MGdnP3J0aW1lPWZmT0RZNkNOMkVn [following]
--2020-11-20 22:05:27--  https://postechackr-my.sharepoint.com/personal/dongbinna_postech_ac_kr/Documents/Research/datasets/counter-fitted-vectors.txt.zip?originalPath=aHR0cHM6Ly9wb3N0ZWNoYWNrci1teS5zaGFyZXBvaW50LmNvbS86dTovZy9wZXJzb25hbC9kb25nYmlubmFfcG9zdGVjaF9hY19rci9FU1Y

* 오리지널 저장소에서 제안한 사용 가능한 단어의 개수가 너무 많으므로 <b>단어의 개수를 줄여 실습</b>합니다.

In [11]:
with open("counter-fitted-vectors.txt", "r") as f:
    words = f.readlines()

# 사용할 단어의 총 개수(약 60,000개)
print("단어의 개수:", len(words))

# 단어의 개수 줄이기: Google Colab 메모리 오류가 나지 않도록 하기(18,000개의 단어만 사용)
target_size = 18000
print("필터링 된 단어의 개수:", target_size)

with open("counter-fitted-vectors.txt", "w") as f:
    f.write(''.join(words[:target_size]))

단어의 개수: 65713
필터링 된 단어의 개수: 18000


#### <b>공격 모델(Targeted Model) 불러오기: BERT</b>

* <b>워드 임베딩(Word Embedding)</b>을 위한 Counter Fitted Vector 텍스트 파일을 다운로드합니다.

In [12]:
!rm -rf ./bert_mr
!mkdir -p ./bert_mr
!wget https://postechackr-my.sharepoint.com/:u:/g/personal/dongbinna_postech_ac_kr/EYuRnUyF9DRKtGxsO--wAqoBG0mIo56jaoJGY4TQwW-8-g?download=1 -O ./bert_mr/pytorch_model.bin
!wget https://postechackr-my.sharepoint.com/:u:/g/personal/dongbinna_postech_ac_kr/ERtJ3QAZl-RDk_iPG8kzf5EBBAPEClPfXknXswTpKA0dvQ?download=1 -O ./bert_mr/bert_config.json
!wget https://postechackr-my.sharepoint.com/:t:/g/personal/dongbinna_postech_ac_kr/EQSBPRwI6AJLheb_KtCOWEsBud5YTH6KtZ5X1srV1q5DDA?download=1 -O ./bert_mr/vocab.txt

--2020-11-20 22:05:37--  https://postechackr-my.sharepoint.com/:u:/g/personal/dongbinna_postech_ac_kr/EYuRnUyF9DRKtGxsO--wAqoBG0mIo56jaoJGY4TQwW-8-g?download=1
Resolving postechackr-my.sharepoint.com (postechackr-my.sharepoint.com)... 40.108.156.33
Connecting to postechackr-my.sharepoint.com (postechackr-my.sharepoint.com)|40.108.156.33|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /personal/dongbinna_postech_ac_kr/Documents/Research/models/BERT%20for%20Movie%20Review/pytorch_model.bin?originalPath=aHR0cHM6Ly9wb3N0ZWNoYWNrci1teS5zaGFyZXBvaW50LmNvbS86dTovZy9wZXJzb25hbC9kb25nYmlubmFfcG9zdGVjaF9hY19rci9FWXVSblV5RjlEUkt0R3hzTy0td0Fxb0JHMG1JbzU2amFvSkdZNFRRd1ctOC1nP3J0aW1lPTY0YnNhYUNOMkVn [following]
--2020-11-20 22:05:38--  https://postechackr-my.sharepoint.com/personal/dongbinna_postech_ac_kr/Documents/Research/models/BERT%20for%20Movie%20Review/pytorch_model.bin?originalPath=aHR0cHM6Ly9wb3N0ZWNoYWNrci1teS5zaGFyZXBvaW50LmNvbS86dTovZy9wZXJzb25hbC9kb25nYmlub

In [13]:
# NLTK 라이브러리 설치
import nltk

nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

#### <b>TextFooler 공격 수행</b>

In [14]:
!python attack_classification.py \
    --dataset_path=./data/mr \
    --target_model=bert \
    --target_model_path=./bert_mr/ \
    --counter_fitting_embeddings_path=counter-fitted-vectors.txt \
    --USE_cache_path=./

2020-11-20 22:06:39.069142: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Data import finished!
Building Model...
Model built!
Building vocab...
Building cos sim matrix...
Start computing the cosine similarity matrix!
tcmalloc: large alloc 2592006144 bytes == 0x11486a000 @  0x7f000548b1e7 0x7f0002ef15e1 0x7f0002f55e88 0x7f0002f56034 0x7f0002f4c96c 0x7f000318947c 0x7f0002fe8e00 0x7f0002fed74a 0x566f73 0x59fd0e 0x7f0002f4170d 0x50a12f 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50beb4 0x5095c8 0x50a2fd 0x50beb4 0x507be4 0x50ad03 0x634e72 0x634f27 0x6386df 0x639281 0x4b0dc0 0x7f0005088bf7 0x5b259a
tcmalloc: large alloc 2592006144 bytes == 0x7efe33f52000 @  0x7f000548b1e7 0x7f0002ef15e1 0x7f0002f55e88 0x7f0002f56034 0x7f0002f4c96c 0x7f000318947c 0x7f0002fe8e00 0x7f0002fed74a 0x566f73 0x59fd0e 0x7f0002f4170d 0x50a12f 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50beb4 0x5095c8 0x50a2fd 0x50beb4 0x507be4 0x50ad03 0x634e72 0x634f27 0x

* <b>공격 결과물(Adversarial Texts)</b>을 출력합니다.
* 정상적으로 분류되는 텍스트 904개 중에서 <b>107개에 대하여 분류 결과를 변경</b>할 수 있습니다.
* 선택 가능한 단어의 수를 줄였기 때문에, 본 결과는 <b>원본 논문보다는 낮은 공격 성공률</b>을 보입니다.
    * 10,000개만 쓸 때의 정확도 변화: 90.400% → 86.500%
    * <b>18,000개</b>만 쓸 때의 정확도 변화: 90.400% → 79.700%

In [15]:
!cat ./adv_results/adversaries.txt

orig sent (0):	davis is so enamored of her own creation that she ca n't see how insufferable the character is
adv sent (1):	davis is so elated of her own creation that she ca n't see how insufferable the character is

orig sent (1):	a muckraking job , the cinematic equivalent of a legal indictment , and a fairly effective one at that
adv sent (0):	a muckraking job , the cinematic equivalent of a legal indictment , and a reasonably reasonable one at that

orig sent (1):	writer director 's mehta 's effort has tons of charm and the whimsy is in the mixture , the intoxicating masala , of cultures and film genres
adv sent (0):	writer director 's mehta 's effort possessed shitloads of charm and the triviality is in the mixture , the bitters masala , of cropping and film genres

orig sent (1):	much of what we see is horrible but it 's also undeniably exceedingly clever
adv sent (0):	keenly of what we see is horrible but it 's also undeniably ridiculously clever

orig sent (1):	rife with the r